# Part 0: Initialising

In [ ]:
# Initializing
import matplotlib.font_manager as font_manager
from matplotlib.lines import Line2D
import matplotlib as mpl
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import re
from scipy.stats import bootstrap
import statsmodels.api as sm
import statsmodels.formula.api as smf
import requests

params = {
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "font.size": 12,
    "legend.fontsize": 12,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "text.usetex": False,
}

mpl.rcParams.update(params)

import warnings

warnings.filterwarnings("ignore")

To start with, we import three datasets that are available on YouNiverse:

``df_timeseries_en.tsv.gz``  
``df_channels_en.tsv.gz``  
``yt_metadata_helper.feather``

In [ ]:
df_time_series = pd.read_csv(
    "./data/df_timeseries_en.tsv.gz", compression="infer", sep="\t"
)
df_time_series["datetime"] = pd.to_datetime(df_time_series["datetime"])
# round the total number of subscribers, it is easier to consider 1 person and instead half of a person...
df_time_series.subs = df_time_series.subs.round(0)

df_channels = pd.read_csv("./data/df_channels_en.tsv.gz", compression="infer", sep="\t")
df_channels["join_date"] = pd.to_datetime(df_channels["join_date"])

In [ ]:
video_metadatas = pd.read_feather(
    "./data/yt_metadata_helper.feather",
    columns=[
        "categories",
        "upload_date",
        "duration",
        "like_count",
        "dislike_count",
        "view_count",
        "channel_id",
    ],
)

In [ ]:
df_time_series.head(2)

In [ ]:
df_channels.head(2)

In [ ]:
video_metadatas.head(2)

We will also import a dataset that is created by us, called `titles_metadata`, which contains the titles and the categories of each video. For Milestone three, we will generate the same metadata for titles, but including the display ID for matching the titles with videos. For the purpose of Milestone 2, we will continue with the current `titles_metadata` for initial analysis.

In [ ]:
titles_metadata = pd.read_feather(
    "./data/yt_metadata_title_helper.feather")

In [ ]:
titles_metadata.head()

# Part 1: Initial Analysis and Data Visualization

In this section, we analyze the YouNiverse data to determine the potential factors that could lead to success on Youtube. This initial analysis will help us narrow down a subset of the dataset to focus on for our Milestone 3 analysis.

## Question 1: How is the weekly content creation pattern of Youtubers regarding the upload frequency, video lengths, upload time?

### Section 1: Video Upload Frequency

In this section, we mainly consider the weekly video upload frequency. To do that, we first group the `video_metadatas` by upload week for all channels, to have an idea on the upload trends. We do not use `df_time_series` for the purpose of this section, which has ready-to-use weekly information regarding each channel, as it only considers the data from 2015-01-05 to 2019-10-06.

In [ ]:
video_count = (
    video_metadatas.groupby(pd.Grouper(key="upload_date", freq="W")).count().channel_id
)

In [ ]:
for year in range(2005, 2021):
    total_videos_year = video_count[video_count.index.year.isin([year])].sum()
    print("We have {:_} video in {}".format(total_videos_year, year))

This data shows that we have to be careful with the different timing within each dataframe used. As we will see later on, there are sharp decreases in the data over time, depending on how many videos were crawled.

In [ ]:
plt.plot(video_count, color="#7570b3", ls="--", label="\% videos uploaded")
plt.yscale("log")
plt.ylabel("Number of Videos")
plt.xlabel("Time")
plt.title("Number of videos uploaded each week")

From the graph above, we can see that the number of videos being uploaded to YouTube has increased steadily since 2005. We want to have a closer look at the number of videos uploaded by each channel to understand how many videos it takes in order to be successfull. For this preliminary analysis, we only consider the number of videos uploaded by each channel per year.

In [ ]:
# The following graph shows how many videos each channel uploads to YouTube per year.
video_metadatas["yearNumber"] = video_metadatas["upload_date"].dt.year
vd_cnt_by_id_yr = (
    video_metadatas.groupby(["channel_id", "yearNumber"])
    .count()
    .categories.unstack()
    .reset_index()
)
vd_cnt_by_id_yr.columns.name = None
vd_cnt_by_id_yr_summary = vd_cnt_by_id_yr.describe()

In [ ]:
plt.fill_between(
    x=list(vd_cnt_by_id_yr_summary.columns.values),
    y1=list(vd_cnt_by_id_yr_summary.loc["25%", :]),
    y2=list(vd_cnt_by_id_yr_summary.loc["75%", :]),
    alpha=0.5,
    color="gray",
)
plt.plot(vd_cnt_by_id_yr_summary.loc["50%", :], color="black")
plt.xlabel("Year")
plt.ylabel("Yearly Upload Frequency")
plt.title("The 2nd and 3rd quartiles of yearly video upload frequency")
# here we choose quartiles, since the mean would be significantly affected by extremely high yearly upload rates

This graphs shows that YouTubers upload more videos every year with time, and the variation of upload frequency between channels also increased significantly since the beginning of the study period.

### Section 2: Video Lengths

Firstly, we will take a look at the basic statistics of video lengths using `video_metadatas` . To make it easier to understand, we first convert the duration into minutes for the subsequent analysis.

In [ ]:
# divide the video duration by 60 to convert duration to minutes
video_metadatas["duration_min"] = video_metadatas["duration"] / 60
video_metadatas.duration_min.describe()

In [ ]:
duration_min_cumul = plt.hist(
    video_metadatas.duration_min, bins=100, log=True, cumulative=-1, histtype="step"
)
plt.title("Histogram of Duration for YouTube Videos (cumulative)")
plt.ylabel("# of Videos (in log scale)")
plt.xlabel("Duration in Minutes")
plt.xticks(rotation=30, ha="right")
plt.show()

In [ ]:
plt.loglog(duration_min_cumul[1][1:], duration_min_cumul[0])
plt.title("Histogram of Duration for YouTube Videos (cumulative)")
plt.ylabel("# of Videos (in log scale)")
plt.xlabel("Duration")
plt.xticks(rotation=30, ha="right")
plt.show()

From the basic statistical information and the histogram, we can observe a significant difference between the mean (12.63 minutes) and the median (4.73 minutes). Furthermore, the maximum duration (100 hours) is much higher compared to both, the median and the mean. The distribution of the duration resembles that of a heavy tail distribution. This observation suggests that there are extremely long videos in the dataset that would increase the mean significantly. Therefore, a visualization with quartiles serves as a less distorted representation of the dataset as it does not consider outliers.

Therefore, we will have a look at the quartiles of duration of videos in the dataset to have a better idea about the distribution: 

In [ ]:
plt.boxplot(video_metadatas["duration_min"], showfliers=False, vert=False)
plt.title("Duration of Videos in Minutes")

Here, we see that throughout the study period, most of the videos are between zero and 25 minutes long. To investigate whether the variation of video length is also varying with time, we take a look at the evolution of this information.

In [ ]:
vd_duration_by_id_yr = (
    video_metadatas.groupby(["channel_id", "yearNumber"])["duration_min"]
    .mean()
    .unstack()
    .reset_index()
)
vd_duration_by_id_yr.columns.name = None
vd_duration_by_id_yr_summary = vd_duration_by_id_yr.describe()

In [ ]:
plt.fill_between(
    x=list(vd_duration_by_id_yr_summary.columns.values),
    y1=list(vd_duration_by_id_yr_summary.loc["25%", :]),
    y2=list(vd_duration_by_id_yr_summary.loc["75%", :]),
    alpha=0.5,
    color="gray",
)
plt.plot(vd_duration_by_id_yr_summary.loc["50%", :], color="black")
plt.xlabel("Year")
plt.ylabel("Average Video Duration for Each Channel")
plt.title("The 2nd and 3rd quartiles of average yearly video duration")
# here we choose quartiles, since the mean would be significantly affected by extremely high yearly upload rates

From this graph, it is obvious that the variance of mean video duration for each channel is increasing with time, and the median itself is also increasing with time. This means that comparing to the beginning of study period, channels increased the video duration, and there are more channels with vastly different video duration towards the end of the study period.

### Section 3: Upload Time
In this section, we want to investigate at what points during the week YouTubers upload their videos.

In [ ]:
video_metadatas["weekNumber"] = video_metadatas["upload_date"].dt.weekday
vd_cnt_by_id_wk = (
    video_metadatas.groupby(["channel_id", "weekNumber", "yearNumber"])
    .count()
    .categories.unstack()
    .reset_index()
)
vd_cnt_by_id_wk.columns.name = None
vd_cnt_by_id_wk["mean_upload"] = vd_cnt_by_id_wk.iloc[:, 2:].mean(axis=1, skipna=True)

In [ ]:
labels = ["MON", "TUE", "WED", "THU", "FRI", "SAT", "SUN"]
ax = sns.boxplot(
    x="weekNumber", y="mean_upload", data=vd_cnt_by_id_wk, showfliers=False
).set(xlabel="Week Days", ylabel="Mean Upload", xticklabels=labels)

This boxplot shows that the variation between upload rate during weekdays are not significantly different. However, during the weekend, it seems that mean upload rate is less than during the week. In the next step, we visualize it better by counting the total number of videos uploaded each day of the week, regardless of the channel ID:

In [ ]:
vd_cnt_by_wk = (
    video_metadatas.groupby(["weekNumber", "yearNumber"])
    .count()
    .categories.unstack()
    .reset_index()
)
vd_cnt_by_wk.columns.name = None

In [ ]:
vd_cnt_by_wk_norm = vd_cnt_by_wk.iloc[:, 2:] / vd_cnt_by_wk.iloc[:, 2:].sum(skipna=True)
plt.rcParams["figure.figsize"] = (15, 10)
plt.plot(vd_cnt_by_wk_norm.T)
plt.legend(
    ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
)
plt.xlabel("year")
plt.ylabel("NORMALISED: video uploaded each weekday")
plt.title("NORMALISED: The evolution of video upload rate for each weekday")
plt.show()

This graph suggests that the number of videos uploaded from Monday to Friday is not significantly different from each other throughout the study period. Generally speaking, YouTubers upload more during weekdays than during the weekends. However, we can also observe an increasing number of videos uploaded on Saturdays in more recent years.

## Question 2: How are the subscription patterns for reaching key numbers of subscribers, aka, 1k, 10k, 100k, 1M?

In this section, we investigate the time frame for YouTubers to reach key milestones in their channel success - number of subscriptions.

In [ ]:
def get_df_channel_ascension(start_subs, end_subs):
    """finds the channels in the time series that have less than the start_subs and more than end_subs.
    It means that it begins with less than start_subs and have now at least end_subs

    Args:
        start_subs (float): low_threshold
        end_subs (float): high_threshold

    Returns:
        dataframe containing all the channels with evolution from starts_subs to end_subs
    """
    under_start_subs = df_time_series[df_time_series["subs"] < start_subs]
    more_end_subs = df_time_series[df_time_series["subs"] > end_subs]
    channel_start_to_end = df_time_series[
        df_time_series.channel.isin(under_start_subs.channel)
        & df_time_series.channel.isin(more_end_subs.channel)
    ]
    return channel_start_to_end

In [ ]:
def get_mean_time_to_reach_X_subs(df, lower_bound, upper_bound):
    time_to_reach_X_subs = df.groupby("channel").apply(
        lambda group: pd.Series(
            {
                "from_zero_to_hero_duration": group[group.subs > upper_bound][
                    "datetime"
                ].iloc[0]
                - group[group.subs < lower_bound]["datetime"].iloc[-1]
            }
        )
    )
    return time_to_reach_X_subs

In [ ]:
channel_10K_to_1M = get_df_channel_ascension(10_000, 1_000_000)
print(
    "We have {} channels that begins with at most 10K subs and reach at least 1M".format(
        channel_10K_to_1M.channel.nunique()
    )
)

Example of a channel to implement the function defined above

In [ ]:
first_channel = channel_10K_to_1M.iloc[0]
test = channel_10K_to_1M[channel_10K_to_1M["channel"] == first_channel.channel]
# test = test[(test['datetime'] > np.datetime64('2018-07-20')) & (test['datetime'] < np.datetime64('2019-01-20'))]
test.plot(x="datetime", y="subs")
print("There are {} weeks for this channel".format(test.shape[0]))

# just check that the first dimension of the test df (it gives the nb of weeks) multiplied
# by 7 (it gives the total nb of days) is equal to the last date - first date. If yes, then we have a continuous
# chronology
print(
    "We should have approximately the same value : \n{} and {}".format(
        7 * test.shape[0], test.iloc[-1].datetime - test.iloc[0].datetime
    )
)

In [ ]:
# get the time it takes to go from less than 10K to 1M
time_to_reach_1M = get_mean_time_to_reach_X_subs(channel_10K_to_1M, 10_000, 1_000_000)
time_to_reach_1M.head()

In [ ]:
print(
    "In average YTbers take {} days to reach 1M of subscribers".format(
        time_to_reach_1M["from_zero_to_hero_duration"].mean().days
    )
)

We will then bootstrap this result for Milestone 3 to see the time it takes to reach 1M subscribers and its confidence interval.

In [ ]:
data = time_to_reach_1M["from_zero_to_hero_duration"].apply(
    lambda delta_time: delta_time.days
)

In [ ]:
rng = np.random.default_rng()
conf_interval_reach_1M = bootstrap(
    (data,),
    np.mean,
    confidence_level=0.95,
    random_state=rng,
).confidence_interval
conf_interval_reach_1M

print(
    "The 95% interval of confidence, the time taken to reach 1M (from 10K) is [{}, {}]".format(
        conf_interval_reach_1M.low, conf_interval_reach_1M.high
    )
)

## Question 3: Which categories are more popular during the study period? Which types of videos receive more positive feedback?

In this question, we will use `video_metadatas` to determine which categories are more popular and have a high amount of views. This step helps in narrowing down our categories of interest to three categories that we want to investigate further for Milestone 3.

### Section 1: Number of Views

In [ ]:
#  make a copy of the video_metadatas
videos_with_cat = video_metadatas[:]
videos_with_cat["categories"] = videos_with_cat["categories"].apply(
    lambda x: x.replace(" & ", "_")
)

# delete the rows with empty categories
videos_with_cat.categories.replace("", np.nan, inplace=True)
videos_with_cat.dropna(subset=["categories"], inplace=True)

In [ ]:
# Average view counts for videos from each category across time
mean_views = (
    videos_with_cat.dropna(axis=0).groupby(["categories", "yearNumber"]).mean()
)
mean_views = mean_views.view_count.unstack().reset_index()
mean_views.columns.name = None
mean_views = mean_views.sort_values(
    by=2019, ascending=False, ignore_index=True
)

print(mean_views.head())

In [ ]:
NUM_COLORS = 15
cm = plt.get_cmap('gist_rainbow')
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_prop_cycle(color=[cm(1.*videos_with_cat.categories.indexof(c)/NUM_COLORS) for c in mean_views.categories])

In [ ]:
# dictionary = [cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)]

In [ ]:
plt.rcParams["figure.figsize"] = (15, 10)
for i in np.arange(mean_views.shape[0]):
    plt.plot(
        mean_views.loc[
            i,
        ][2:],
        label=mean_views.loc[i,][
            0:1
        ][0],
    )
plt.semilogy()    
plt.legend(loc="upper right")
plt.xlabel("year")
plt.ylabel("mean video view count")
plt.title("mean video view counts per year for each category")
plt.show()

The results of the data above is obtained by averaging the view counts of all videos in the same category by year. It shows that on average, Music and Comedy are the two top categories for view counts from 2005 to 2017. From 2018 to 2019, Film & Animation replaces Music on the top two spots. However, all top four categories are showing a decrease in their mean video view counts. However, there is not a genre of videos that shows obvious growth in mean video view counts over the study period, while most of the categories seem to stay more or less the same level, non-profit and news & politics categories seem to have obvious drop in their mean video view counts.

Now let's investigate how popular are the videos from each category, and how the trend changes over time. For this purpose, we will now shift to the total video view counts, instead of the mean ones.

In [ ]:
# determine how many videos are uploaded to YouTube by each channel by year
video_count_by_year = (
    videos_with_cat.dropna(axis=0).groupby(["categories", "yearNumber"]).sum()
)
video_view_count_by_year = video_count_by_year.view_count.unstack().reset_index()
video_view_count_by_year.columns.name = None
video_view_count_by_year = video_view_count_by_year.sort_values(
    by=2019, ascending=False, ignore_index=True
)
# NOTE THAT CATEGORIES::MOVIES AND SHOWS HAVE VERY FEW DATA!

In [ ]:
plt.rcParams["figure.figsize"] = (15, 10)
for i in np.arange(video_view_count_by_year.shape[0]):
    plt.plot(
        video_view_count_by_year.loc[
            i,
        ][2:],
        label=video_view_count_by_year.loc[i,][
            0:1
        ][0],
    )
plt.semilogy()    
plt.legend(loc="lower right")
plt.xlabel("year")
plt.ylabel("video view count")
plt.title("video view counts per year for each category")
plt.show()

From previous plot, we could see that by 2019, the top four categories with highest mean view counts are Comedy, Film & Animation, Music, Gaming, whereas People & Blogs was placed at the 7th. However, here we can observe that the total view counts for People & Blogs now enters the top four. This means that although that videos under People & Blogs have on average less views per video, there are probably a lot of videos under People & Blogs. Let's verify this observation by another plot.


In [ ]:
# determine how many videos are uploaded to YouTube by each channel by year
upload_count_by_year = (
    videos_with_cat.dropna(axis=0).groupby(["categories", "yearNumber"]).count()
)
upload_count_by_year = upload_count_by_year.view_count.unstack().reset_index()
upload_count_by_year.columns.name = None
upload_count_by_year = upload_count_by_year.sort_values(
    by=2019, ascending=False, ignore_index=True
)

plt.rcParams["figure.figsize"] = (15, 10)
for i in np.arange(upload_count_by_year.shape[0]):
    plt.plot(
        upload_count_by_year.loc[
            i,
        ][2:],
        label=upload_count_by_year.loc[i,][
            0:1
        ][0],
    )
plt.semilogy()    
plt.legend(loc="lower right")
plt.xlabel("year")
plt.ylabel("video upload count")
plt.title("upload counts per year for each category")
plt.show()

Here we can observe that indeed there are more videos under people & Blogs category, compared to the other two categories with high mean view counts, Comedy and Film & Animation. This result suggests that the high total view counts can beat Comedy and Film & Animation probably because of its high number of videos uploaded.

Lastly, we want to find out how are the video views distributed between different categories for each year. To do that, we will divide the total video counts per category per year by total video counts per year.

In [ ]:
# NORMALISATION:
## 1. view count - normalise wrt. total video views per year, to see which categories is viewed more every year
plt.rcParams["figure.figsize"] = (15, 10)
for i in np.arange(video_view_count_by_year.shape[0]):
    plt.plot(
        video_view_count_by_year.loc[
            i,
        ][2:]
        / video_view_count_by_year.sum(axis=0)[2:],
        label=video_view_count_by_year.loc[i,][
            0:1
        ][0],
    )
plt.semilogy()  
plt.legend(loc="lower right")
plt.xlabel("year")
plt.ylabel("NORMALISED: video view count")
plt.title("NORMALISED: video view counts per year for each category")
plt.show()
# view on music videos decreases, whereas on entertainment and gaming increases

The result indicates that the share of video counts across different categories are quite stable across the study period. However, it is also obvious that the view counts for Music is decreasing, and that for Entertainment, Gaiming and People & Blogs are somewhat increasing.

TO-DO: ADD NOTES

In [ ]:
# use the time series dataframe to implement this, it would be easier. 
df_time_series['year'] = df_time_series.datetime.dt.year
# penalize the big channel (many subs) by dividing the number of delta views by nb of total subscribers
df_time_series['views_over_subs'] = df_time_series.delta_views / df_time_series.subs
df_time_series.sample(4)

In [ ]:
df_time_series[df_time_series.delta_views == 0].category.value_counts()

In [ ]:
views_over_subs_per_year = (
    df_time_series.groupby(["category", "year"]).sum()
)
views_over_subs_per_year.head(10)

In [ ]:
views_over_subs_per_year = views_over_subs_per_year.views_over_subs.unstack().reset_index()
views_over_subs_per_year.columns.name = None
views_over_subs_per_year

In [ ]:
NUM_COLORS = 15
cm = plt.get_cmap('gist_rainbow')
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])
plt.rcParams["figure.figsize"] = (15, 10)
for i in np.arange(views_over_subs_per_year.shape[0]):
    plt.plot(
        views_over_subs_per_year.loc[
            i,
        ][2:],
        label=views_over_subs_per_year.loc[i,][
            0:1
        ][0],
    )
    
plt.legend(loc="upper left")
plt.xlabel("year")
plt.ylabel("video view count divided by number of subs")
plt.title("video view counts divided by nb of subs per year for each category")
plt.show()

### Section 2: Number of Likes and Dislikes

In [ ]:
video_like_count_by_year = video_count_by_year.like_count.unstack().reset_index()
video_like_count_by_year.columns.name = None
video_like_count_by_year = video_like_count_by_year.sort_values(
    by=2019, ascending=False, ignore_index=True
)


In [ ]:
video_dislike_count_by_year = video_count_by_year.dislike_count.unstack().reset_index()
video_dislike_count_by_year.columns.name = None
video_dislike_count_by_year = video_dislike_count_by_year.sort_values(
    by=2019, ascending=False, ignore_index=True
)

In [ ]:
## 3. like count versus view count - normalise wrt. total like counts over view counts for each category for each year
plt.rcParams["figure.figsize"] = (15, 10)
for i in np.arange(video_like_count_by_year.shape[0]):
    plt.plot(
        video_like_count_by_year.loc[
            i,
        ][2:]
        / (video_view_count_by_year.loc[i][2:]),
        label=video_like_count_by_year.loc[i,][
            0:1
        ][0],
    )
plt.semilogy()
plt.legend(loc="lower right")
plt.xlabel("year")
plt.ylabel("NORMALISED: like over views")
plt.title("NORMALISED: video like counts over view counts per category per year")
plt.show()


In [ ]:
plt.rcParams["figure.figsize"] = (15, 10)
for i in np.arange(video_dislike_count_by_year.shape[0]):
    plt.plot(
        video_dislike_count_by_year.loc[
            i,
        ][2:]
        / (video_view_count_by_year.loc[i][2:]),
        label=video_dislike_count_by_year.loc[i,][
            0:1
        ][0],
    )
plt.semilogy()  
plt.legend(loc="lower right")
plt.xlabel("year")
plt.ylabel("video dislikes count")
plt.title("video dislikes counts per year for each category")
plt.show()


## Question 4: What type of titles attract more attention?
Here, we want to do an initial analysis on the titles by analyzing the positive/negative tones and personal pronouns used in them, and how does it change from category to category. For Milestone 3, we want to analyse the changing patterns of titles for the three categories we want to focus on.

### Section 1: Number of Words

In [ ]:
def count_words_simple(x):
    return len(x.split(" "))

In [ ]:
# remove all rows with no category label
titles_metadata.categories.replace("", np.nan, inplace=True)
titles_metadata.dropna(subset=["categories"], inplace=True)

titles_metadata['title'] = titles_metadata['title'].apply(lambda x : x.lower())
titles_metadata['numwords'] = titles_metadata['title'].apply(lambda text : count_words_simple(text.strip()))
titles_metadata.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
sns.boxplot(x="categories", y="numwords", data=titles_metadata, showfliers=False)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
ax.set_ylabel("number of words in title")
plt.show()

This graph indicates that the title lengths for videos across different categories are not significantly different from each other. That being said, there are some variations, such as that videos under Comedy seem to have less number of words, whereas videos under Entertainment, Gaming, Sports and News & Politics seems to have more words.

### Section 2: Positive and Negative Sentiments

In [ ]:
# Import the positive words and negative words database
POSITIVE_URL = 'https://ptrckprry.com/course/ssd/data/positive-words.txt'
NEGATIVE_URL = 'https://ptrckprry.com/course/ssd/data/negative-words.txt'

def get_words(URL):
    r = requests.get(URL)
    data = r.text
    words = []
    for line in data.split('\n'):
        if ';' not in line and len(line) != 0:
            words.append(line)
    return words

positive_words = get_words(POSITIVE_URL)
negative_words = get_words(NEGATIVE_URL)

# compare the words in the headline with the words in the list of pos and neg words otherwise it is too expansive 
def get_columns_pos_neg(list_words, df, colname):
    new_column = []
    for row in df[colname]:
        # get all the words in the headline without ponctuation to be ready to be compared with the neg and pos words  
        row_words = re.sub(r'[^\w\s]', '', row).split(' ')
        if any(word in list_words for word in row_words):
            new_column.append(1)
        else :
            new_column.append(0)      
    return new_column

In [ ]:
# we will first create a copy of titles_metadata with 200000 samples randomly chosen, as running the entire dataset
# would take too much time
titles_metadata_sample = titles_metadata.iloc[
    np.random.choice(np.arange(titles_metadata.shape[0]), 200000), ].reset_index(drop = True)

In [ ]:
# add positive and negative score for each title
titles_metadata_sample['positive'] = get_columns_pos_neg(positive_words, titles_metadata_sample, 'title')
titles_metadata_sample['negative'] = get_columns_pos_neg(negative_words, titles_metadata_sample, 'title')

In [ ]:
# count among those video titles containing positive or negative words from our database,
# how is the share between positive and negative titles
pos_neg_ratio = titles_metadata_sample.groupby("categories").sum()
pos_neg_ratio.drop('numwords', inplace=True, axis=1)
pos_neg_ratio = pos_neg_ratio/(np.column_stack(pos_neg_ratio.positive + pos_neg_ratio.negative).T)

In [ ]:
# plot
barWidth = 0.85
categories = pos_neg_ratio.index.values.tolist()
r = np.arange(len(categories))
# Create red Bars
plt.bar(r, pos_neg_ratio['negative'], bottom=[i for i in pos_neg_ratio['positive']], color='#F15C6C', 
        edgecolor='white', width=barWidth, label = "negative")
# Create green Bars
plt.bar(r, pos_neg_ratio['positive'], color='#69BF97', edgecolor='white', width=barWidth, label = "positive")

# Custom x axis
plt.xticks(r, categories)
plt.xlabel("Categories")
plt.ylabel("Share of Videos")
plt.ylim(0,1)
plt.legend(loc='upper left', bbox_to_anchor=(1,1), ncol=1)
plt.xticks(rotation=90)
# Show graphic
plt.show()

Here, we can see that in Comedy, Gaming and News & Politics, there more more negative video titles, Autos & Vehicles, Entertainment and Filmd & Amination have more or less the same share, whereas the rest have more positive video titles than negative ones.

### Section 3: Personal Pronouns

In [ ]:
# (this section of code is already provided)
feature_wordsets = dict([
    # https://en.wikipedia.org/wiki/English_personal_pronouns
    ('first_person_singular', ['i', 'me', 'my', 'mine', 'myself',
                               "i'd", "i'll", "i'm", "i've", 'id', 'im', 'ive']),
    ('first_person_plural', ['we', 'us', 'our', 'ours', 'ourselves',
                              "we'd", "we'll", "we're", "we've",]),
    ('second_person', ['you','your','yours','yourself',
                              "ya", "you'd", "you'll", "you're", "you've", 'youll', 'youre', 'youve', 
                              'yourselves']),
    ('third_person_singular', ['he','him','his','himself',
                               "he'd", "he's", 'hes',
                               'she','her','hers','herself', 
                               "she'll", "she's", 'shes',
                               'it','its','itself',
                               'themself']),
    ('third_person_plural', ['they','them','their','theirs','themselves',
                              "they'd", "they'll", "they've", 'theyll', 'theyve'])
])

(feature_wordsets.keys())

In [ ]:
def get_new_columns_pronouns(feature_wordsets, df, colname):
    dict_pronoun_list = dict([
        ('first_person_singular', []),
        ('first_person_plural', []),
        ('second_person', []),
        ('third_person_singular', []),
        ('third_person_plural', []),
    ])
    for headline in df[colname]:
        for key in feature_wordsets.keys():
            # if a word corresponding to a pronoun is present in the headline, 
            #then add 1 to this pronoun list (column) otherwise add 0 
            if any(re.search(r'\b' + pronoun + r'\b', headline) for pronoun in feature_wordsets[key]):
                dict_pronoun_list[key].append(1)
            else :
                dict_pronoun_list[key].append(0)
    return dict_pronoun_list

dict_pronoun_list = get_new_columns_pronouns(feature_wordsets, titles_metadata_sample, 'title')

# create the new columns in our dataframe
for col in dict_pronoun_list.keys():
    titles_metadata_sample[col] = dict_pronoun_list[col]

In [ ]:
# count among those video titles containing personal pronouns from our database,
# how is the share between each personal pronoun group
per_pro_ratio = titles_metadata_sample.groupby("categories").sum()
per_pro_ratio.drop(['numwords','positive', 'negative'], inplace=True, axis=1)
per_pro_ratio = per_pro_ratio/(np.column_stack(per_pro_ratio.first_person_singular + 
                                               per_pro_ratio.first_person_plural +
                                               per_pro_ratio.second_person + 
                                               per_pro_ratio.third_person_singular + 
                                               per_pro_ratio.third_person_plural).T)

In [ ]:
# plot
barWidth = 0.85
categories = per_pro_ratio.index.values.tolist()
r = np.arange(len(categories))

plt.bar(r, per_pro_ratio['third_person_plural'], 
        bottom=[i+j+k+l for i,j,k,l in zip(per_pro_ratio['first_person_singular'], per_pro_ratio['first_person_plural'], 
                                           per_pro_ratio['second_person'], per_pro_ratio['third_person_singular'])], 
        color='#8C68AB', edgecolor='white', width=barWidth, label = "third_person_plural")

plt.bar(r, per_pro_ratio['third_person_singular'], 
        bottom=[i+j+k for i,j,k in zip(per_pro_ratio['first_person_singular'], 
                                       per_pro_ratio['first_person_plural'], per_pro_ratio['second_person'])], 
        color='#E68A99', edgecolor='white', width=barWidth, label = "third_person_singular")

plt.bar(r, per_pro_ratio['second_person'], 
        bottom=[i+j for i,j in zip(per_pro_ratio['first_person_singular'], per_pro_ratio['first_person_plural'])], 
        color='#C1CF94', edgecolor='white', width=barWidth, label = "second_person")

plt.bar(r, per_pro_ratio['first_person_plural'], 
        bottom=[i for i in per_pro_ratio['first_person_singular']], 
        color='#F5E3C1', edgecolor='white', width=barWidth, label = "first_person_plural")

plt.bar(r, per_pro_ratio['first_person_singular'], 
        color='#B6B6B6', edgecolor='white', width=barWidth, label = "first_person_singular")

# Custom x axis
plt.xticks(r, categories)
plt.xlabel("Categories")
plt.ylabel("Share of Videos")
plt.ylim(0,1)
plt.legend(loc='upper left', bbox_to_anchor=(1,1), ncol=1)
plt.xticks(rotation=90)
# Show graphic
plt.show()

This shows that the use of personal pronouns varies significantly between different categories. For example, we would expect News & Politics to use less first personal pronouns and more third personal pronouns, since the subjects are often inpersonal and general, whereas for Travel & Events, and People & Blogs, we would expect the video titles to use more first personal pronouns, since the videos are often personal. Both expectations are reflected on the plotting.

# Part 2: Further Analysis For Milestone 3

In this section, we will focus on three specific categories and determine what are the factors that could lead to more subscriptions. As a reminder, our job in the project is to better inform our young 'brother' who would like to become an YouTuber. Therefore, we will focus on categories that is achievable by just one person, aka, eliminating categories such as Entertainment, News & Politics, Film & Animation, Movies and Shows. Our young 'brother' is not particularly talented in Music, Handcraft, so we will also eliminate Howto & Style, and Music. Since he is still in High School, he is not yet able to explain explain science or teach others, we will also exclude Science & Technology, and Education. From previous graphs, we would recommend him to consider XX, XX and XX.

## Question 1: How does the video upload frequency, time of the week, and video length affect the subscription rate of the channels?


## Question 2: How does the language used in titles affect subscription number?

## Question 3:  Can we predict the channel's success based on channel information, including average video length, upload frequency, usual time of uploads, categories of videos uploaded, positive/negative sentiments of the title, usual way of addressing the viewers, the number of words in the title, and the number of tags used.

## Question 4: 